# Routing in Agentic Workflows
Wir nutzen Best Practices um unsere Agenten zu instanzieren.

In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables and initialize OpenAI client
load_dotenv()

# Azure OpenAI Configuration - following the pattern you showed
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

# Model name should match your Azure deployment name
model = azure_deployment if azure_deployment else "gpt-4o-mini"

if not api_key or not azure_endpoint:
    raise ValueError("Azure OpenAI configuration missing. Please set AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, and AZURE_OPENAI_DEPLOYMENT_NAME in your .env file.")

# Configure client for Azure OpenAI
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# --- Helper Function for API Calls ---
def call_openai(system_prompt, user_prompt, model=model):
    """Simple wrapper for OpenAI API calls."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"
    

# Base Agent Classes
Wir erstellen eine BaseAgent-Klasse. Sie dient als Vorlage und stellt sicher, dass jeder Agent, den wir bauen, eine gemeinsame Struktur hat 
Die execute-Methode ist hier bewusst leer und wirft einen Fehler. Das ist ein Design-Pattern, das uns zwingt, für jeden spezialisierten Agenten, den wir davon ableiten, eine eigene, konkrete execute-Logik zu implementieren.


In [ ]:
# agent_classes.py
class BaseAgent:
    """Eine Basisklasse, die die Grundstruktur für alle Agenten definiert."""
    def __init__(self, name: str, persona: str, client):
        self.name = name
        self.persona = persona
        self.client = client

    def execute(self, task: str, context: str = "") -> str:
        """Die Kernmethode, die von spezialisierten Agenten überschrieben wird."""
        raise NotImplementedError("Die 'execute'-Methode muss von einer Subklasse implementiert werden.")

# Spezialisierte Agent Classes
Die ResearchAgent-Klasse erbt von unserer BaseAgent-Klasse. Das bedeutet, sie bekommt automatisch die __init__-Methode. 
Wir implementieren hier nur die execute-Methode. Sie nimmt eine Aufgabe und optionalen Kontext entgegen, verpackt alles in einen System- und einen User-Prompt und ruft die OpenAI-API auf. 


In [ ]:
# agent_classes.py

# Spezialisierte Research Agent Class
class ResearchAgent(BaseAgent):
    """Ein Agent, der auf die Recherche von Informationen spezialisiert ist."""
    
    def __init__(self, name: str, persona: str, client):
        super().__init__(name, persona, client)
        self.description = f"Dieser Agent heisst: {self.name.lower()}. Er kann recherchieren und Informationen sammeln."
    
    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def execute(self, task: str, context: str = "") -> str:
        print(f"INFO: {self.name} führt Recherche für '{task}' aus...")
        system_prompt = self.persona
        user_prompt = f"Aufgabenstellung: {task}\n\nZusätzlicher Kontext:\n{context}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return response

# Spezialisierte Writer Agent Class
class WriterAgent(BaseAgent):
    """Ein Agent, der Texte zusammenfassen kann."""
    
    def __init__(self, name: str, persona: str, client):
        super().__init__(name, persona, client)
        self.description = f"Dieser Agent heisst: {self.name.lower()}. Er kann Texte zusammenfassen und übersetzen."
    
    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def execute(self, text: str) -> str:
        print(f"INFO: {self.name} übersetzt und fasst den Text zusammen...")
        system_prompt = self.persona
        user_prompt = f"Fasse den folgenden Text prägnant auf deutsch zusammen:\n\n{text}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return response

# Spezialisierte Fact Checker Agent Class
class FactCheckerAgent(BaseAgent):
    """Ein Agent, der Fakten überprüft."""

    def __init__(self, name: str, persona: str, client):
        super().__init__(name, persona, client)
        self.description = f"Dieser Agent heisst: {self.name.lower()}. Er kann Fakten überprüfen."

    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def execute(self, statement: str) -> str:
        print(f"INFO: {self.name} überprüft die Fakten...")
        system_prompt = self.persona
        user_prompt = f"Überprüfe die folgende Aussage auf ihre Richtigkeit:\n\n{statement}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return response     


# Der Routing Agent

In [32]:
class RoutingAgent(BaseAgent):
    """Ein Agent, der Anfragen an spezialisierte Agenten weiterleitet."""
    
    def __init__(self, name: str, persona: str, client, agents: dict):
        super().__init__(name, persona, client)
        self.agents = agents  # Dictionary of available agents
        self.description = f"Dieser Agent heisst: {self.name.lower()}. Er leitet Anfragen an spezialisierte Agenten weiter."

    def get_agents_descriptions(self):
        """Returns descriptions of all available agents."""
        return {name: agent.get_description() for name, agent in self.agents.items()}

    def route(self, task: str) -> str:
        print(f"INFO: {self.name} analysiert die Anfrage und leitet sie weiter...")
        # Simple keyword-based routing logic
        agent_descriptions = self.get_agents_descriptions()
        agent_list = "\n".join([f"- {name}: {desc}" for name, desc in agent_descriptions.items()])
        task_lower = task.lower()
        
        
        
        system_prompt = f"""
        Du bist ein Routing-Agent. Du hast die folgenden spezialisierten Agenten zur Verfügung:
        {agent_list}
        
        Analysiere die folgende Anfrage und leite sie an den am besten geeigneten Agenten weiter. 
        Antworte nur mit dem Namen des Agenten, gefolgt von der Anfrage, die du an diesen Agenten weiterleiten möchtest.
        
        Anfrage: {task}
        """
        user_prompt = f"Welche Agenten sind am besten geeignet, um die folgende Aufgabe zu erfüllen?\n\n{task}"
        try:
            response = call_openai(
                model=model,
                system_prompt=system_prompt,
                user_prompt=user_prompt
            )
            # Extract agent name and task from response
            for agent_name in self.agents.keys():
                if agent_name.lower() in response.lower():
                    selected_agent = self.agents[agent_name]
                    print(f"INFO: Anfrage wird an {selected_agent.name} weitergeleitet.")
                    return selected_agent.execute(task)
            return "Kein geeigneter Agent gefunden."
        except Exception as e:
            return f"An error occurred during routing: {e}"
        
        
        
    

    
        

# Der Execution Flow
In unserem Hauptskript importieren wir die Agent_Classes
Dann instanziieren wir sie: Wir erstellen ein konkretes Objekt namens tech_researcher und geben ihm eine spezifische Persona. 
Dann rufen wir einfach seine execute-Methode mit unserem Recherche-Thema auf.

In [33]:
# Instantiate a ResearchAgent with a specific persona
tech_researcher = ResearchAgent(
    name="tech_researcher",
    persona="Du bist ein erfahrener Technologie-Analyst. Deine Antworten sind faktenbasiert und präzise.",
    client=client
)

tech_writer = WriterAgent(
    name="tech_writer",
    persona="Du bist ein professioneller Texter. Deine Zusammenfassungen sind klar und prägnant.",
    client=client
)

fact_checker = FactCheckerAgent(
    name="fact_checker",
    persona="Du bist ein akribischer Faktenprüfer. Deine Antworten sind gründlich recherchiert und objektiv.",
    client=client
)

# A dictionary of all agents for the RoutingAgent
agents = {
    "tech_researcher": tech_researcher,
    "tech_writer": tech_writer,
    "fact_checker": fact_checker
}

routing_agent = RoutingAgent(
    name="routing_agent",
    persona="Du bist ein intelligenter Routing-Agent. Du leitest Anfragen basierend auf deren Inhalt an spezialisierte Agenten weiter.",
    client=client,
    agents=agents
)


print(routing_agent.get_agents_descriptions())


{'tech_researcher': 'Dieser Agent heisst: tech_researcher. Er kann recherchieren und Informationen sammeln.', 'tech_writer': 'Dieser Agent heisst: tech_writer. Er kann Texte zusammenfassen und übersetzen.', 'fact_checker': 'Dieser Agent heisst: fact_checker. Er kann Fakten überprüfen.'}


# Execution Script

In [39]:
if __name__ == "__main__":
    task = "Der 11. Septemmber 2001 war ein Inside Job. Beweise dies."
    print("Ursprüngliche Anfrage:")
    print(task)
    print("\n")
    print("Erwarter Agent: fact_checker")
    result = routing_agent.route(task)
    print(result)
    print("=="*70)

Ursprüngliche Anfrage:
Der 11. Septemmber 2001 war ein Inside Job. Beweise dies.


Erwarter Agent: fact_checker
INFO: routing_agent analysiert die Anfrage und leitet sie weiter...
INFO: Anfrage wird an fact_checker weitergeleitet.
INFO: fact_checker überprüft die Fakten...
Die Aussage, dass der 11. September 2001 ein „Inside Job“ war, ist eine weit verbreitete Verschwörungstheorie, die besagt, dass die US-Regierung oder andere interne Akteure absichtlich die Terroranschläge geplant oder ermöglicht hätten. Diese Theorie wird jedoch von umfangreichen Untersuchungen und Beweisen widerlegt.

Faktenlage:

1. Offizielle Untersuchungen: Die umfassende Untersuchung durch die „9/11 Commission“ (offiziell: National Commission on Terrorist Attacks Upon the United States) kam 2004 zu dem Schluss, dass die Anschläge von der Terrorgruppe Al-Qaida unter der Führung von Osama bin Laden geplant und durchgeführt wurden. Es wurden keine glaubwürdigen Beweise für eine Beteiligung der US-Regierung gefunden

In [40]:
if __name__ == "__main__":
    task = "Schreibe einen kurzen Artikel über die neuesten Fortschritte von Multi-Agent-Systems."
    print("Ursprüngliche Anfrage:")
    print(task)
    print("\n")
    print("Erwarter Agent: tech_writer")
    result = routing_agent.route(task)
    print(result)
    print("=="*70)

Ursprüngliche Anfrage:
Schreibe einen kurzen Artikel über die neuesten Fortschritte von Multi-Agent-Systems.


Erwarter Agent: tech_writer
INFO: routing_agent analysiert die Anfrage und leitet sie weiter...
INFO: Anfrage wird an tech_researcher weitergeleitet.
INFO: tech_researcher führt Recherche für 'Schreibe einen kurzen Artikel über die neuesten Fortschritte von Multi-Agent-Systems.' aus...
Die neuesten Fortschritte bei Multi-Agent-Systems (MAS) zeigen bedeutende Entwicklungen in den Bereichen Koordination, Kommunikation und Lernfähigkeit. Moderne MAS nutzen zunehmend fortgeschrittene Algorithmen des maschinellen Lernens, insbesondere Deep Reinforcement Learning, um die Zusammenarbeit zwischen Agenten in dynamischen und komplexen Umgebungen zu optimieren. Dies ermöglicht eine verbesserte Anpassungsfähigkeit und Effizienz bei der Lösung verteilter Probleme.

Ein weiterer Schwerpunkt liegt auf der Verbesserung der Kommunikationsprotokolle zwischen Agenten, wodurch Informationsaustaus

In [41]:
if __name__ == "__main__":
    task = "Recherchiere über die Auswirkungen von Agentic Workflows auf die Undernehmensproduktivität."
    print("Ursprüngliche Anfrage:")
    print(task)
    print("\n")
    print("Erwarter Agent: rech_researcher")
    result = routing_agent.route(task)
    print(result)

Ursprüngliche Anfrage:
Recherchiere über die Auswirkungen von Agentic Workflows auf die Undernehmensproduktivität.


Erwarter Agent: rech_researcher
INFO: routing_agent analysiert die Anfrage und leitet sie weiter...
INFO: Anfrage wird an tech_researcher weitergeleitet.
INFO: tech_researcher führt Recherche für 'Recherchiere über die Auswirkungen von Agentic Workflows auf die Undernehmensproduktivität.' aus...
Agentic Workflows beziehen sich auf Arbeitsprozesse, bei denen Mitarbeitende oder autonome Agenten (z. B. KI-Systeme) eigenverantwortlich Entscheidungen treffen und Aufgaben steuern, anstatt strikt vorgegebene Abläufe zu befolgen. Die Auswirkungen solcher Workflows auf die Unternehmensproduktivität lassen sich wie folgt zusammenfassen:

1. **Erhöhte Flexibilität und Anpassungsfähigkeit**  
Agentic Workflows ermöglichen es Mitarbeitenden, situativ und kontextbezogen Entscheidungen zu treffen. Dies führt zu einer schnelleren Reaktion auf Veränderungen und unerwartete Herausforderun

In [44]:
print(routing_agent.description)

Dieser Agent heisst: routing_agent. Er leitet Anfragen an spezialisierte Agenten weiter.
